## Example analysis notebook
Just some example code to automate analysis on all files in a folder using tools in this repo.
The example script below is written assuming you have two cameras, with all videos in the same folder. Videos from one camera start with `cam1` and videos from the other start with `cam2`, with exactly the same remaining file names.
That folder should also contain the `DLTcoefs.csv` file, and must include `DLTcoefs` in the file name.
As written, the second code cell runs `deeplabcut.analyze_videos` on all videos in that folder. If they are already analyzed, that step will be skipped (or you can comment out that line)
It will also create labeled videos for each video.

Because these files are not (yet) pip installable:
1. Clone the github repo to a folder on your computer. DLC should be installed on the computer.
2. Run the notebook in jupyter using the python kernel in your DLC env.
3. Change the entries in the next cell.
4. Run both code cells.
5. Enjoy the results.

Please cite the Argus paper if you use this code because this depends heavily on that repository.

In [ ]:
# ANALYZE AND RECONSTRUCT IN A FOLDER
like=0.85
usefiltered = True # True or False
#set your path to config
config = r"C:\Users\...\config.yaml"

#set the path to the folder you want to analyze that contains all videos
folderpath = r"C:\Users\...\videos"

# set the video type
videotype = 'avi'

In [ ]:
# Don't change anything below here unless you want to change settings
import deeplabcut as dlc
from pathlib import Path
import dlt2dlclabels as dtod
import dlc2dlt as dlctod
import tools

# find cam1 vids
cam1vids = list(Path(folderpath).glob(f"*cam1*.{videotype}"))
dlt = list(Path(folderpath).glob("*DLTcoefs*"))[0]

# use DLC functions to get scorer part of file names
cfg = dlc.auxiliaryfunctions.read_config(config)

# analyze with DLC
scorer = dlc.analyze_videos(config, [folderpath], videotype=videotype, gputouse=0, save_as_csv=True)

if usefiltered:
    dlc.filterpredictions(config, [folderpath], videotype=videotype, save_as_csv=True)

# make a labled video (can comment out if not needed or change settings)
dlc.create_labeled_video(config, [folderpath], videotype=videotype, filtered = usefiltered, draw_skeleton=True)

# convert back to dlt format and triangulate
for c1 in cam1vids:
    opath = str(c1.parent / c1.stem[5:]) + f"_DLC2DLT_iter{cfg['iteration']}"
    c2 = Path(str(c1).replace("cam1", "cam2"))
    if not c2.is_file():
        print(f"{c2.name} cannot be found, check to make sure it is named precisely like camera 1")
        continue
    # get dlc tracks for each video...need dlcscorer
    if usefiltered:
        c1data = str(c1.parent / c1.stem) + scorer + "_filtered.h5"
        c2data = str(c2.parent / c2.stem) + scorer + "_filtered.h5"
    else:
        c1data = str(c1.parent / c1.stem) + scorer + ".h5"
        c2data = str(c2.parent / c2.stem) + scorer + ".h5"
    
    #convert dlc tracks to dlt format
    dlctod.dlc2dlt(config, opath, [c1data, c2data], flipy=False, offsets = [0,0], like=like, videotype='.avi')
    
    #triangulate the points to make xyzpts files for each trial
    tools.triangulate(opath + "-xypts.csv", dlt, flipy=False, profpath=None)